In [1]:
import json
import os

import random



In [18]:
pretrain_path = f"data_pretrain/train"
finetune_path = f"anon_data/train/"
citation_path = f"citation_data/"

## Load data

In [12]:
with open(os.path.join(pretrain_path,f"pile_mistral_v0.1_train_val.json"),'r') as f:
    data_pile = json.load(f)

In [13]:
with open(os.path.join(pretrain_path,f"urchadesynthetic-pii-ner-mistral-v1_train_val.json"),'r') as f:
    data_pii = json.load(f)

In [6]:
with open(os.path.join(finetune_path,f"train_empty.json"),'r') as f:
    anon_data_train_empty = json.load(f)

In [7]:
with open(os.path.join(finetune_path,f"train_non_empty.json"),'r') as f:
    anon_data_train_non_empty = json.load(f)

In [9]:
with open(os.path.join(finetune_path,f"validation_empty.json"),'r') as f:
    anon_data_val_empty = json.load(f)

In [ ]:
with open(os.path.join(finetune_path,f"validation_non_empty.json"),'r') as f:
    anon_data_val_non_empty = json.load(f)

In [19]:
with open(os.path.join(citation_path,f"citation_train.json"),'r') as f:
    citation_data = json.load(f)

In [16]:
print(f"size pile-mistral :         {len(data_pile)}")
print(f"size pii :                  {len(data_pii)}")
print(f"size anon train empty :     {len(anon_data_train_empty)}")
print(f"size anon train n-empty :   {len(anon_data_train_non_empty)}")
print(f"size anon val empty :       {len(anon_data_val_empty)}")
print(f"size anon val n-empty :     {len(anon_data_train_non_empty)}")



size pile-mistral :         17751
size pii :                  17671
size anon train empty :     564961
size anon train n-empty :   303458


NameError: name 'anon_data_val_empty' is not defined

# Dataset mix = 1

Data_pile-mistral : 90% of full dataset for training, 10% for testing -> 17.5K
Data_pii : 90% of full dataset for training, 10% for testing -> 17.5K 
--> total pretrain data : 35K
anon_data_train_non_empty:  randomly sample 10K
anon_data_train_empty: randomly sample 10K
citation_data: randomly sample 20K 


In [20]:
random.seed(42)

merge_v1 = data_pile + data_pii + random.sample(anon_data_train_non_empty,10**4) + random.sample(anon_data_train_empty,10**4) + random.sample(citation_data,2*10**4)

In [22]:
#save merge_v1 dataset for finetuning

with open(f"finetune_data/merged_v1.json",'w') as f:
    json.dump(merge_v1,f)